In [3]:
!pip install -q datasets transformers peft accelerate trl

In [2]:
!pip install --upgrade fsspec==2024.12.0 gcsfs==2024.12.0


  Attempting uninstall: gcsfs
    Found existing installation: gcsfs 2025.3.2
    Uninstalling gcsfs-2025.3.2:
      Successfully uninstalled gcsfs-2025.3.2


In [3]:
from google.colab import drive

# Replace with the actual path to your JSON file in Google Drive
gdrive_json_path = '/content/drive/MyDrive/Asha_llm_data/women_jobs_chatbot_data.json'

In [4]:
import json
from datasets import Dataset

try:
    with open(gdrive_json_path, 'r') as f:
        data = json.load(f)
    dataset = Dataset.from_list(data)
    print(f"✅ Dataset loaded with {len(dataset)} samples from Google Drive.")
    print(f"Sample dataset entry:\n{dataset[0]}")
except FileNotFoundError:
    print(f"❌ Error: JSON file not found at '{gdrive_json_path}'. Please check the path.")
    exit()
except json.JSONDecodeError:
    print(f"❌ Error: Could not decode JSON from '{gdrive_json_path}'. Please ensure it's a valid JSON file.")
    exit()

✅ Dataset loaded with 4694 samples from Google Drive.
Sample dataset entry:
{'instruction': 'Answer women job related general queries. Tell me about the job: Research Intern – Generative AI Agents', 'output': 'Company: synergylabs, Location: gurugram, haryana, india, Description: job title research intern generative ai agent location gurugram duration month stipend inr u technologydriven startup exploring future aipowered agent automation mission build scalable solution using cuttingedge generative ai llm autonomous agent solve realworld problem join u youre curious innovative ready get hand dirty ai research role overview research intern dive deep world generative ai agent large language model llm autonomous multiagent system youll help u map landscape explore technical capability evaluate tool contribute prototype development key responsibility conduct comprehensive research generative ai agent llmbased agent autonomous agent framework analyze recent paper blog github project industr

In [5]:
!pip install -U bitsandbytes
!pip install -U accelerate
!pip install -U transformers
!pip install -U peft


  Using cached transformers-4.51.0-py3-none-any.whl.metadata (38 kB)
Using cached transformers-4.51.0-py3-none-any.whl (10.4 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.3
    Uninstalling transformers-4.50.3:
      Successfully uninstalled transformers-4.50.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 9.1 MB/s eta 0:00:00
  Attempting uninstall: peft
    Found existing installation: peft 0.14.0
    Uninstalling peft-0.14.0:
      Successfully uninstalled peft-0.14.0


In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from peft import get_peft_model, LoraConfig, TaskType

model_id = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_id)
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# Apply LoRA for efficient fine-tuning
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM,
)
model = get_peft_model(base_model, peft_config)
model.print_trainable_parameters()


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

trainable params: 884,736 || all params: 248,462,592 || trainable%: 0.3561


In [9]:
print("--- Inspecting Dataset ---")
print(dataset)
print(dataset[0])

--- Inspecting Dataset ---
Dataset({
    features: ['instruction', 'output'],
    num_rows: 4694
})
{'instruction': 'Answer women job related general queries. Tell me about the job: Research Intern – Generative AI Agents', 'output': 'Company: synergylabs, Location: gurugram, haryana, india, Description: job title research intern generative ai agent location gurugram duration month stipend inr u technologydriven startup exploring future aipowered agent automation mission build scalable solution using cuttingedge generative ai llm autonomous agent solve realworld problem join u youre curious innovative ready get hand dirty ai research role overview research intern dive deep world generative ai agent large language model llm autonomous multiagent system youll help u map landscape explore technical capability evaluate tool contribute prototype development key responsibility conduct comprehensive research generative ai agent llmbased agent autonomous agent framework analyze recent paper blo

In [10]:
def preprocess(example):
    instruction = example["instruction"].strip()
    output = example["output"].strip()

    # Optional: Add custom prompt formatting
    example["instruction"] = f"Instruction: {instruction}"
    example["output"] = f"Response: {output}"

    return example

# Apply before tokenizing
cleaned_dataset = dataset.map(preprocess)


Map:   0%|          | 0/4694 [00:00<?, ? examples/s]

In [13]:
max_input_len = 512
max_target_len = 128

def tokenize(example):
    input_enc = tokenizer(example['instruction'], padding="max_length", truncation=True, max_length=max_input_len)
    target_enc = tokenizer(example['output'], padding="max_length", truncation=True, max_length=max_target_len)
    input_enc["labels"] = target_enc["input_ids"]
    return input_enc

tokenized_dataset = dataset.map(tokenize, remove_columns=["instruction", "output"])


Map:   0%|          | 0/4694 [00:00<?, ? examples/s]

In [14]:
from transformers import AutoModelForSeq2SeqLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

# Configure LoRA
lora_config = LoraConfig(
    r=8,                # rank
    lora_alpha=32,
    target_modules=["q", "v"],  # for T5, use q and v projection layers
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 884,736 || all params: 248,462,592 || trainable%: 0.3561


In [16]:
training_args = TrainingArguments(
    output_dir="./flan-t5-lora-model",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=1,
    save_steps=100,
    report_to="none",
    label_names=["labels"]  # 👈 Add this line
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

trainer.train()


<ipython-input-16-a9a7d64fca65>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss


KeyboardInterrupt: 